# 🤖 Introduction aux Large Language Models (LLMs)

## 🎯 Objectifs

Dans ce notebook, nous allons découvrir :

- 🧠 **Qu'est-ce qu'un LLM** ? (ChatGPT, Claude, GPT-4)
- 🏗️ **L'architecture Transformer** - La révolution de 2017
- 🔤 **Comment les LLMs "comprennent" le texte**
- 📊 **GPT vs BERT vs T5** - Différentes approches
- 🛣️ **Notre parcours d'apprentissage** - Ce qu'on va construire

---

## 💡 Qu'est-ce qu'un Large Language Model ?

### Définition Simple

Un **LLM** est un réseau de neurones entraîné sur **énormément de texte** pour :

1. **Comprendre** le langage naturel
2. **Générer** du texte cohérent
3. **Accomplir** des tâches variées (traduction, résumé, code, etc.)

### Exemples Concrets

| Modèle | Entreprise | Paramètres | Release |
|--------|-----------|------------|----------|
| **GPT-4** | OpenAI | ~1.76T | 2023 |
| **Claude 3** | Anthropic | Non divulgué | 2024 |
| **Gemini** | Google | ~1.5T | 2024 |
| **LLaMA 2** | Meta | 70B | 2023 |
| **Mistral** | Mistral AI | 7B | 2023 |

**Notre mini-LLM** : ~10 millions de paramètres (éducatif)

---

## 🏛️ Histoire : L'Évolution vers les LLMs

### Ligne du Temps

```
1950s  : Premiers systèmes basés sur des règles
1980s  : Réseaux de neurones récurrents (RNN)
1997   : LSTM (Long Short-Term Memory)
2013   : Word2Vec (embeddings)
2017   : 🔥 TRANSFORMER (révolution)
2018   : BERT, GPT-1
2019   : GPT-2
2020   : GPT-3 (emergence)
2022   : ChatGPT (explosion)
2023   : GPT-4, LLaMA, Claude 2
2024   : Claude 3, Gemini, modèles open-source
```

### Le Moment Charnière : 2017

📜 **Paper** : "Attention Is All You Need" (Vaswani et al.)

**Innovation** : L'architecture **Transformer** remplace les RNN/LSTM

**Pourquoi c'est révolutionnaire ?**

| RNN/LSTM | Transformer |
|----------|-------------|
| ❌ Séquentiel (lent) | ✅ Parallèle (rapide) |
| ❌ Mémoire limitée | ✅ Attention sur tout le contexte |
| ❌ Difficile à entraîner | ✅ Plus stable |
| ❌ Max ~1000 tokens | ✅ Jusqu'à 128k tokens (GPT-4) |

---

In [ ]:
# Imports pour ce notebook
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)

print("✓ Prêt à explorer les LLMs !")

## 🧩 Comment Fonctionne un LLM ? (Vue d'Ensemble)

### Le Pipeline Complet

```
INPUT: "Le chat mange une"

┌─────────────────────────────────────────────────────┐
│  1️⃣ TOKENIZATION                                    │
│     Texte → Nombres                                 │
│     "Le chat mange une" → [142, 2304, 15673, 1739]  │
└─────────────────────────────────────────────────────┘
            ↓
┌─────────────────────────────────────────────────────┐
│  2️⃣ EMBEDDINGS                                      │
│     Nombres → Vecteurs denses                       │
│     142 → [0.23, -0.45, 0.67, ...] (256 dimensions) │
└─────────────────────────────────────────────────────┘
            ↓
┌─────────────────────────────────────────────────────┐
│  3️⃣ POSITIONAL ENCODING                            │
│     Ajouter l'info de position                      │
│     "Le"(pos=0), "chat"(pos=1), ...                 │
└─────────────────────────────────────────────────────┘
            ↓
┌─────────────────────────────────────────────────────┐
│  4️⃣ TRANSFORMER LAYERS (×N)                        │
│     ┌──────────────────────────────────┐            │
│     │  Multi-Head Self-Attention       │ ← MAGIE !  │
│     │  (contexte complet)              │            │
│     └──────────────────────────────────┘            │
│     ┌──────────────────────────────────┐            │
│     │  Feed-Forward Network            │            │
│     │  (transformations)               │            │
│     └──────────────────────────────────┘            │
│     (Répété 4-96 fois selon le modèle)              │
└─────────────────────────────────────────────────────┘
            ↓
┌─────────────────────────────────────────────────────┐
│  5️⃣ OUTPUT PROJECTION                              │
│     Vecteurs → Probabilités sur vocabulaire         │
│     [0.45, 0.23, 0.12, ...] (50k probabilités)      │
└─────────────────────────────────────────────────────┘
            ↓
┌─────────────────────────────────────────────────────┐
│  6️⃣ SAMPLING                                        │
│     Choisir le prochain token                       │
│     Probabilité max → "souris" (token 4523)         │
└─────────────────────────────────────────────────────┘

OUTPUT: "Le chat mange une souris"
```

---

## 🔑 Le Concept Clé : ATTENTION

### Analogie Simple

Imagine que tu lis cette phrase :

> "Le chat de Marie est mignon. **Il** aime jouer."

**Question** : À quoi "Il" fait-il référence ?

**Humain** : Je regarde le contexte → "Il" = "le chat"

**LLM avec Attention** : Calcule l'importance de chaque mot précédent pour comprendre "Il"

### Visualisation de l'Attention

```
Mot actuel : "Il"

Regarde le contexte :
"Le"     →  5%  (peu important)
"chat"   → 80%  (très important) ✓
"de"     →  2%  (peu important)
"Marie"  → 10%  (un peu important)
"est"    →  1%  (peu important)
"mignon" →  2%  (peu important)
```

Le modèle "attend" (attend to) principalement sur "chat" !

---

In [ ]:
# Démonstration simplifiée de l'attention

# Phrase exemple
words = ["Le", "chat", "de", "Marie", "est", "mignon", "Il"]

# Scores d'attention (simulés) pour le mot "Il" regardant les mots précédents
# En réalité, ces scores sont calculés par le modèle
attention_scores = np.array([0.05, 0.80, 0.02, 0.10, 0.01, 0.02])

# Visualisation
fig, ax = plt.subplots(figsize=(12, 6))

colors = ['lightblue' if score < 0.5 else 'salmon' for score in attention_scores]
bars = ax.barh(words[:-1], attention_scores, color=colors, edgecolor='black', linewidth=2)

# Ajouter les valeurs
for i, (bar, score) in enumerate(zip(bars, attention_scores)):
    ax.text(score + 0.02, bar.get_y() + bar.get_height()/2, 
            f'{score:.0%}', 
            va='center', fontsize=12, fontweight='bold')

ax.set_xlabel('Score d\'Attention', fontsize=12, fontweight='bold')
ax.set_ylabel('Mots du Contexte', fontsize=12, fontweight='bold')
ax.set_title('Attention du mot "Il" sur le contexte précédent', 
             fontsize=14, fontweight='bold')
ax.set_xlim([0, 1])
ax.grid(axis='x', alpha=0.3)

# Annotation
ax.annotate('"Il" se concentre surtout sur "chat" !', 
            xy=(0.80, 1), xytext=(0.5, 3),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=12, color='red', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 C'est ça l'ATTENTION : le modèle apprend automatiquement")
print("   sur quels mots se concentrer pour comprendre le contexte !")

---

## 📐 Architectures de Transformers : GPT vs BERT vs T5

### Les 3 Grandes Familles

#### 1️⃣ **Decoder-Only** (GPT family)

```
Structure : Decoder seulement
Task     : Génération de texte (autorégressif)
Masking  : Causal (ne voit que le passé)

Exemples : GPT-2, GPT-3, GPT-4, LLaMA, Mistral

Usage    : Chatbots, génération créative, completion
```

**Notre projet** : On va construire un modèle **GPT-style** ! 🎯

#### 2️⃣ **Encoder-Only** (BERT family)

```
Structure : Encoder seulement
Task     : Compréhension (bidirectionnel)
Masking  : Bidirectionnel (voit passé ET futur)

Exemples : BERT, RoBERTa, ALBERT

Usage    : Classification, NER, Q&A
```

#### 3️⃣ **Encoder-Decoder** (T5 family)

```
Structure : Encoder + Decoder
Task     : Traduction, résumé

Exemples : T5, BART, mT5

Usage    : Traduction, résumé, reformulation
```

### Comparaison Visuelle

```
GPT (Decoder-Only)
─────────────────────
Input:  "Le chat mange"
        ↓
     [Decoder] → Masked Self-Attention (causal)
        ↓
Output: "une souris" (génération)


BERT (Encoder-Only)
────────────────────
Input:  "Le [MASK] mange une souris"
        ↓
     [Encoder] → Self-Attention (bidirectionnel)
        ↓
Output: "chat" (prédiction du masque)


T5 (Encoder-Decoder)
────────────────────
Input:  "Traduire en anglais: Le chat mange"
        ↓
     [Encoder] → Context
        ↓
     [Decoder] → Generation
        ↓
Output: "The cat eats"
```

---

## 🎓 Pourquoi GPT pour notre Projet ?

### Raisons Pédagogiques

1. ✅ **Plus simple** : Une seule composante (decoder)
2. ✅ **Plus intuitif** : Génération de texte est facile à comprendre
3. ✅ **Plus populaire** : ChatGPT, Claude = GPT-style
4. ✅ **Plus fun** : On peut discuter avec notre modèle !

### Architecture GPT Simplifiée

```python
class MiniGPT:
    def __init__(self):
        self.token_embedding = Embedding(vocab_size, d_model)
        self.position_embedding = PositionalEncoding(max_len, d_model)

        self.transformer_blocks = [
            TransformerBlock(d_model, n_heads)  # ← On va construire ça !
            for _ in range(n_layers)
        ]

        self.output_projection = Linear(d_model, vocab_size)

    def forward(self, x):
        # 1. Embeddings
        x = self.token_embedding(x) + self.position_embedding(x)

        # 2. Transformer blocks
        for block in self.transformer_blocks:
            x = block(x)  # ← Magie de l'attention ici !

        # 3. Projection
        logits = self.output_projection(x)

        return logits
```

**C'est ce qu'on va implémenter from scratch !**

---

## 🗺️ Notre Parcours d'Apprentissage

### Étape par Étape

#### Phase 1 : Fondamentaux (Notebooks 00-03)

1. **00 - Introduction** ← Tu es ici !
2. **01 - Tokenization** : Texte → Nombres
   - BPE (Byte-Pair Encoding)
   - Construction du vocabulaire
   - Encode/Decode

3. **02 - Embeddings** : Nombres → Vecteurs riches
   - Word2Vec concepts
   - Embedding layers
   - Similarité sémantique

4. **03 - Attention** : Le cœur du Transformer
   - Queries, Keys, Values
   - Scaled Dot-Product Attention
   - Implémentation from scratch

#### Phase 2 : Architecture (Notebooks 04-07)

5. **04 - Multi-Head Attention** : Plusieurs perspectives
6. **05 - Positional Encoding** : Ordre des mots
7. **06 - Transformer Block** : Assembler les pièces
8. **07 - GPT Complet** : Le modèle final

#### Phase 3 : Training & Generation (Notebooks 08-11)

9. **08 - Dataset** : Préparer les données
10. **09 - Training** : Entraîner le modèle
11. **10 - Generation** : Générer du texte
12. **11 - Fine-tuning** : Adapter à des tâches

#### Phase 4 : Projet Final (Notebook 12)

13. **12 - Mini-ChatGPT** : Projet complet end-to-end

---

## 📊 Comparaison : Notre Mini-GPT vs GPT-3

### Spécifications

| Paramètre | Notre Mini-GPT | GPT-3 | GPT-4 |
|-----------|----------------|-------|-------|
| **Vocabulaire** | ~5,000 tokens | 50,257 tokens | ~100k tokens |
| **Embedding dim** | 256 | 12,288 | ~18,000 |
| **Layers** | 4-6 | 96 | ~120 |
| **Attention heads** | 8 | 96 | ~128 |
| **Context window** | 128 tokens | 2,048 tokens | 128k tokens |
| **Paramètres** | **~10M** | **175B** | **~1.76T** |
| **Dataset** | 1MB texte | 45TB texte | Inconnu |
| **Coût d'entraînement** | ~0€ (CPU) | ~$4.6M | ~$100M |
| **Temps d'entraînement** | Minutes/Heures | Semaines | Mois |

### Objectifs Réalistes

**Notre mini-GPT pourra** :
- ✅ Générer du texte dans le style de l'entraînement (ex: Shakespeare)
- ✅ Compléter des phrases
- ✅ Suivre des patterns simples
- ✅ Démontrer la compréhension de l'attention

**Il NE pourra PAS** :
- ❌ Rivaliser avec ChatGPT
- ❌ Raisonner de manière complexe
- ❌ Suivre des instructions variées
- ❌ Connaissances du monde réel étendues

**C'est un projet PÉDAGOGIQUE** - L'objectif est la **compréhension** ! 🎓

---

In [ ]:
# Visualisation : Échelle des paramètres

models = ['Notre\nMini-GPT', 'GPT-2\nSmall', 'GPT-2\nLarge', 'GPT-3', 'GPT-4\n(estimé)']
params = [10e6, 117e6, 1.5e9, 175e9, 1760e9]  # En paramètres
params_billions = [p / 1e9 for p in params]  # Convertir en milliards

fig, ax = plt.subplots(figsize=(12, 6))

colors = ['lightgreen', 'skyblue', 'orange', 'salmon', 'red']
bars = ax.bar(models, params_billions, color=colors, edgecolor='black', linewidth=2)

# Échelle log pour mieux voir
ax.set_yscale('log')
ax.set_ylabel('Nombre de Paramètres (Milliards)', fontsize=12, fontweight='bold')
ax.set_title('Comparaison de la Taille des Modèles', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Ajouter les valeurs
for bar, p in zip(bars, params_billions):
    height = bar.get_height()
    if p < 1:
        label = f'{p*1000:.0f}M'
    else:
        label = f'{p:.0f}B'

    ax.text(bar.get_x() + bar.get_width()/2., height,
            label,
            ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Notre modèle est 17,500× plus petit que GPT-3 !")
print("   Mais il nous permet de COMPRENDRE les concepts fondamentaux.")

---

## 🧪 Exemple Concret : Comment GPT Génère du Texte

### Le Processus Autorégressif

GPT génère un token à la fois, en utilisant tous les tokens précédents comme contexte.

```python
# Pseudo-code simplifié

prompt = "Il était une fois"
generated = prompt

for _ in range(50):  # Générer 50 tokens
    # 1. Encoder le texte actuel
    tokens = tokenize(generated)

    # 2. Forward pass dans le modèle
    logits = model(tokens)

    # 3. Obtenir les probabilités du prochain token
    probs = softmax(logits[-1])  # Dernier token

    # 4. Échantillonner (sample) le prochain token
    next_token = sample(probs, temperature=0.8)

    # 5. Ajouter au texte généré
    generated += decode(next_token)
```

### Exemple Pas à Pas

```
Prompt initial : "Il était une fois"

Itération 1:
  Contexte : "Il était une fois"
  Prédictions : {"un": 0.35, "une": 0.25, "le": 0.15, ...}
  Choisi : "un"
  Texte : "Il était une fois un"

Itération 2:
  Contexte : "Il était une fois un"
  Prédictions : {"roi": 0.40, "prince": 0.20, "dragon": 0.10, ...}
  Choisi : "roi"
  Texte : "Il était une fois un roi"

Itération 3:
  Contexte : "Il était une fois un roi"
  Prédictions : {"qui": 0.50, ",": 0.20, "nommé": 0.15, ...}
  Choisi : "qui"
  Texte : "Il était une fois un roi qui"

... et ainsi de suite !
```

**Key insight** : Le modèle utilise TOUT le contexte précédent grâce à l'attention ! 🎯

---

## 💪 Pourquoi Construire from Scratch ?

### On pourrait juste utiliser Hugging Face...

```python
# 2 lignes et c'est fait !
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('gpt2')
```

### Mais construire from scratch permet de :

1. ✅ **Comprendre en profondeur** comment ça marche
2. ✅ **Démystifier la "magie"** de l'attention
3. ✅ **Pouvoir déboguer** et améliorer
4. ✅ **Innover** sur de nouvelles architectures
5. ✅ **Impressionner** en entretien technique 😎

### Philosophie de ce Cours

> "Si tu ne peux pas le construire from scratch,  
> tu ne le comprends pas vraiment."

**Notre approche** :
1. Phase 1-3 : **NumPy** (from scratch complet)
2. Phase 4 : **PyTorch** (scaling up)
3. Comparaison : Notre implémentation vs **Transformers library**

---

## 📚 Ressources Complémentaires

### Papers Fondamentaux

1. **"Attention Is All You Need"** (Vaswani et al., 2017)
   - Le paper original du Transformer
   - https://arxiv.org/abs/1706.03762

2. **"Improving Language Understanding by Generative Pre-Training"** (Radford et al., 2018)
   - GPT-1
   - Introduction du pretraining + fine-tuning

3. **"Language Models are Unsupervised Multitask Learners"** (Radford et al., 2019)
   - GPT-2
   - Zero-shot learning

4. **"Language Models are Few-Shot Learners"** (Brown et al., 2020)
   - GPT-3
   - Emergence de capacités

### Tutoriels et Blogs

- **The Illustrated Transformer** (Jay Alammar)
  - http://jalammar.github.io/illustrated-transformer/
  - Visualisations excellentes

- **The Annotated Transformer** (Harvard NLP)
  - http://nlp.seas.harvard.edu/annotated-transformer/
  - Code ligne par ligne

- **Andrej Karpathy - "Let's build GPT"**
  - YouTube : Construction step-by-step

### Livres

- **"Speech and Language Processing"** (Jurafsky & Martin)
- **"Natural Language Processing with Transformers"** (Tunstall et al.)

---

## 🎯 Résumé et Prochaines Étapes

### Ce qu'on a appris

✅ **Qu'est-ce qu'un LLM** : Réseau de neurones entraîné sur du texte  
✅ **L'architecture Transformer** : Révolution de 2017  
✅ **Le concept d'Attention** : Regarder le contexte pertinent  
✅ **GPT vs BERT vs T5** : Différentes approches  
✅ **Notre objectif** : Construire un mini-GPT from scratch  

### Concepts Clés à Retenir

1. 🔑 **Attention** : Le cœur du Transformer
2. 🎯 **Autorégressif** : Génère un token à la fois
3. 🧱 **Composantes** : Embeddings → Transformer → Projection
4. 📊 **Échelle** : Des millions aux billions de paramètres

### Prochaine Étape

**Notebook 01 - Tokenization** 🎯

On va apprendre comment transformer du texte en nombres !

```
"Bonjour le monde" → [145, 298, 1023]
```

**Pourquoi c'est important ?**
- Les réseaux de neurones ne comprennent que les nombres
- Le choix du tokenizer impacte les performances
- GPT utilise BPE (Byte-Pair Encoding)

---

## 🎓 Exercices (Optionnels)

1. **Recherche** : Trouve la taille (en paramètres) de LLaMA 2, Mistral 7B, Claude 3
2. **Réflexion** : Pourquoi GPT-4 a besoin de 128k tokens de contexte ?
3. **Exploration** : Essaye ChatGPT et observe comment il génère du texte token par token
4. **Lecture** : Lis "The Illustrated Transformer" de Jay Alammar

---

**Prêt pour la suite ? Let's go ! 🚀**

**→ Notebook suivant : `01_tokenization.ipynb`**